## Simulation for Multi-Lane NTT/INTT

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sympy.ntheory.residue_ntheory import nthroot_mod
import os
import time
from multiprocessing import Pool

In [2]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

In [3]:
def modinv(a, m):
    '''
    input (a, m)
    a: input
    m: Modulus
    '''
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('Modular inverse does not exist')
    else:
        return x % m

In [4]:
# Bit-Reverse integer
def bit_reverse(a, n):
    return int(('{:0'+str(n)+'b}').format(a)[::-1],2)

def indexReverse(B, v):
    '''
    B : NTT result in bit-reverse order (BO)
    '''
    n = len(B)
    reversed_indices = [0] * n

    for i in range(n):
        reversed_indices[i] = bit_reverse(i, v) # int(format(i, '0' + str(v) + 'b')[::-1], 2)

    result = [0] * n
    for i in range(n):
        result[reversed_indices[i]] = B[i]

    return result

# # Example usage
# B = [0, 1, 2, 3]
# v = 2

# reversed_B = indexReverse(B, v)
# print(reversed_B)

In [5]:
def tfg(n, q):
    if nthroot_mod(-1,n,q) != None:
        psi = int(nthroot_mod(-1,n,q))
        # print("generate psi table...")
        Y_table = [0] * n  # Start with the first element, which is 1 (psi^0 mod q)
        Y_table[0] = 1
        
        for i in range(1, n):
            Y_table[i] = (psi * Y_table[i-1]) % q
    else:
        print("nthroot_mod not found.")

    return Y_table

In [6]:
def tfg_table(psi, n, q):
    # print("generate psi table...")
    Y_table = [0] * n  # Start with the first element, which is 1 (psi^0 mod q)
    Y_table[0] = 1
    
    for i in range(1, n):
        Y_table[i] = (psi * Y_table[i-1]) % q

    return Y_table

In [7]:
# Cooley-Tukey Butterfly Structure
# A0,A1: input coefficients
# W: twiddle factor
# q: modulus
# B0,B1: output coefficients

# JIT-compiled butterfly
# @njit
def CT_Butterfly(A0,A1,W,q):
    r"""
    A0 -------\--|+|-- B0
               \/
               /\
    A1 --|x|--/--|-|-- B1
    """
    M = (A1 * W) % q

    B0 = (A0 + M) % q
    B1 = (A0 - M) % q

    return B0,B1

In [8]:
# Gentleman-Sandle Butterfly Structure
# A0,A1: input coefficients
# W: twiddle factor
# q: modulus
# B0,B1: output coefficients
def GS_Butterfly(A0,A1,W,q):
    r"""
    A0 --\--|+|------- B0
          \/
          /\
    A1 --/--|-|--|x|-- B1
    """
    M0 = (A0 + A1) % q
    M1 = (A0 - A1) % q

    B0 = M0
    B1 = (M1 * W) % q

    return B0,B1

In [9]:
def DIV2(n,q):
    if (n % 2 == 0):
        n = n >> 1  # Right-shift to divide by 2
    else: # n is odd
        n = (n >> 1) + ((q + 1) >> 1)   # Modular adjustment for odd n
    return n

In [10]:
def GS_BU_DIV2(A0,A1,W,q):
    B0, B1 = GS_Butterfly(A0,A1,W,q)
    return DIV2(B0,q),DIV2(B1,q)

In [11]:
def parallel_CT_layer(coeff_pairs, twiddle_factors, q):
    '''
    Process 16 butterfly operations in parallel
    Input: 32 coefficients (16 pairs), 16 twiddle factors
    Output: 32 processed coefficients
    This simulates one cycle of 16 parallel BUs in hardware
    
    Args:
        coeff_pairs: list of tuples [(a0,b0), (a1,b1), ..., (a15,b15)] - 16 pairs of coefficients
        twiddle_factors: list of 16 twiddle factors [W0, W1, ..., W15]
        q: modulus
        
    Returns:
        list of 32 processed coefficients
    '''
    BU_COUNT = 16
    results = [None] * (BU_COUNT * 2)  # 32 output coefficients
    
    # Process 16 butterfly operations in parallel (simulating hardware)
    for bu_idx in range(BU_COUNT):
        a_val = coeff_pairs[bu_idx][0]
        b_val = coeff_pairs[bu_idx][1]
        W = twiddle_factors[bu_idx]
        
        # Each BU processes one butterfly operation
        result = CT_Butterfly(a_val, b_val, W, q)
        
        # Store results (in hardware, this would happen simultaneously)
        results[bu_idx * 2] = np.asarray(result[0]).item()
        results[bu_idx * 2 + 1] = np.asarray(result[1]).item()
        
    return results

In [12]:
# --- NTT functions ---
def NTT(A, Y_table, q, debug=False):
    '''
    Iterative Radix-2 Cooley-Tukey Number Theoretic Transform (NTT)
    We refer to negacyclic convolution
    A       : Polynomial in coefficient form (COEF form) in normal order
    q       : Modulus
    Y_table : contains TF (2nth root of unity) constants in bit-reverse order, 
    hatA    : bit-reverse order (EVAL form / NTT form)
    '''
    n = len(A)        # n - 1, Degree of polynomial,  which is the highest power of the variable in polynomial
    hatA = A.copy()   # hatA = NTT(A)

    v = math.floor(math.log2(n))    # number of stages

    t = n >> 1
    m = 1  
  
    while m < n:
        for i in range(m):
            idx_omega = m + i
            # print("loop",idx_omega)
            W = Y_table[idx_omega]
            for a_idx in range(i * 2**v, (i * 2**v) + t):
                if debug:
                    print(hatA[a_idx], hatA[a_idx + t])
                hatA[a_idx], hatA[a_idx + t] = CT_Butterfly(hatA[a_idx], hatA[a_idx + t], W, q)
                if debug:
                    print(hatA[a_idx], hatA[a_idx + t])
        if debug:
            print(hatA)

        v -= 1
        t = t//2
        m = m*2

    return hatA

In [ ]:
def NTT_parallel_16BU(A, Y_table, q):
    """
    Perform the Number Theoretic Transform (NTT) using 16 parallel Butterfly Units (BUs).
    Parallel CT-NTT using precomputed twiddle table Y_table

    Parameters:
        A       : Polynomial in coefficient form (COEF form) in normal order
        q       : Modulus
        Y_table : Twiddle factors (2n-th roots of unity) in bit-reversed order

    Returns:
        hatA    : Transformed polynomial in Evaluation Form (EVAL/NTT form)
    """
    n = 65536
    stages = 16
    hatA = A.copy()
    total_ops = n // 2  # 32768
    num_chunks = total_ops // 16  # 2048
    
    # Main NTT processing
    for stage in range(stages):
        m = 1 << stage              # Number of groups (2^stage)
        t = n >> (stage + 1)        # Distance between elements in the current stage

        # Process butterfly operations in chunks of 16 (simulating 16 parallel BUs)
        for chunk_id in range(num_chunks):
            active_BUs = 16
            
            # Prepare inputs for this chunk of parallel operations
            coeff_pairs = []
            twiddle_factors = []
            
            # Collect data for active BUs in this chunk
            for bu_id in range(active_BUs):
                op_idx = (chunk_id << 4) + bu_id

                # Map op_idx to (group, offset) correctly
                i = op_idx // t          # Which group
                offset = op_idx % t      # Which pair within group

                # Compute butterfly indices
                a_idx = i * (2 * t) + offset
                a_idx_plus_t = a_idx + t

                # Twiddle factor
                idx_omega = m + i
                twiddle_idx = bit_reverse(idx_omega, 16)

                coeff_pairs.append((hatA[a_idx], hatA[a_idx_plus_t]))
                twiddle_factors.append(Y_table[twiddle_idx])
            
            # Process 16 butterfly operations in parallel
            processed_coeffs = parallel_CT_layer(coeff_pairs, twiddle_factors, q)
            
            # Apply results back to the main array
            for bu_id in range(active_BUs):
                op_idx = (chunk_id << 4) + bu_id
                i = op_idx // t
                offset = op_idx % t
                a_idx = i * (2 * t) + offset
                a_idx_plus_t = a_idx + t
                hatA[a_idx] = processed_coeffs[bu_id * 2]
                hatA[a_idx_plus_t] = processed_coeffs[bu_id * 2 + 1]

    return hatA

In [14]:
# Reading inverse psi table from file
psi_table = pd.read_csv("psi_partq_df.csv")
psi_table = psi_table.to_numpy(dtype=object)

In [15]:
# Reading modulus
qp = pd.read_csv("qp.csv")
qp = qp.to_numpy(dtype=object).flatten()  # Flatten the DataFrame to a 1D array
qp_partp = qp[24:32]


In [16]:
mu = (1 << (48*2)) // qp
mu = np.array(mu, dtype=object)

In [17]:
qp_partq = qp[0:24]
qp_partp = qp[24:32]

In [18]:
moddown_part1 = pd.read_csv('moddown_part1.csv')
moddown_part1 = moddown_part1.to_numpy(dtype=object)  # Flatten the DataFrame to a 1D array

### NTT

Simulation for 1 lane NTT

In [47]:
# # Convert bconv result to EVAL form
# moddown_part1_eval = [[0] for _ in range(len(moddown_part1))]
# for i in range(len(moddown_part1)):
#     a = moddown_part1[i] # COEF form
#     qi = qp_partq[i]
#     psi = psi_table[i].tolist()
#     result = NTT_parallel_16BU(a, indexReverse(psi, int(math.log2(len(psi)))), qi)
#     moddown_part1_eval[i] = result
# moddown_part1_eval = np.array(moddown_part1_eval, dtype=object)
# pd.DataFrame(moddown_part1_eval)

In [45]:
EVAL_a = moddown_part1[0] # EVAL form
psi = psi_table[0].tolist()

In [58]:
def NTT_parallel_16BU(A, Y_table, q):
    """
    Perform the Number Theoretic Transform (NTT) using 16 parallel Butterfly Units (BUs).
    Parallel CT-NTT using precomputed twiddle table Y_table

    Parameters:
        A       : Polynomial in coefficient form (COEF form) in normal order
        q       : Modulus
        Y_table : Twiddle factors (2n-th roots of unity) in bit-reversed order

    Returns:
        hatA    : Transformed polynomial in Evaluation Form (EVAL/NTT form)
    """
    n = 65536
    stages = 16
    hatA = A.copy()
    total_ops = n // 2  # 32768
    num_chunks = total_ops // 16  # 2048
    
    # Main NTT processing
    for stage in range(stages):
        m = 1 << stage              # Number of groups (2^stage)
        t = n >> (stage + 1)        # Distance between elements in the current stage
        t_mask = t - 1

        # print(f"Stage {stage}")

        # Process butterfly operations in chunks of 16 (simulating 16 parallel BUs)
        for chunk_id in range(num_chunks):
            active_BUs = 16

            # print(f"Chunk {chunk_id}")
            
            # Prepare inputs for this chunk of parallel operations
            coeff_pairs = []
            twiddle_factors = []

            idx_coeff_pairs = []
            idx_twiddle_factors = []
            
            # Collect data for active BUs in this chunk
            for bu_id in range(active_BUs):
                op_idx = (chunk_id << 4) + bu_id

                # Map op_idx to (group, offset) correctly
                i = op_idx // t          # Which group
                offset = op_idx & t_mask     # Which pair within group

                # Compute butterfly indices
                a_idx = i * (2 * t) + offset
                a_idx_plus_t = a_idx + t

                # Twiddle factor
                idx_omega = m + i
                twiddle_idx = bit_reverse(idx_omega, 16)

                coeff_pairs.append((hatA[a_idx], hatA[a_idx_plus_t]))
                twiddle_factors.append(Y_table[twiddle_idx])

                idx_coeff_pairs.append((a_idx, a_idx_plus_t))
                idx_twiddle_factors.append(twiddle_idx)

            # --- Process 16 butterfly operations in parallel
            # This simulates one cycle of 16 parallel BUs in hardware
            # Prepare inputs for this chunk of parallel operations
            # print(idx_twiddle_factors)
            
            # Process 16 butterfly operations in parallel
            processed_coeffs = parallel_CT_layer(coeff_pairs, twiddle_factors, q)
            
            # Apply results back to the main array
            for bu_id in range(active_BUs):
                op_idx = (chunk_id << 4) + bu_id
                i = op_idx // t
                offset = op_idx & t_mask
                a_idx = i * (2 * t) + offset
                a_idx_plus_t = a_idx + t
                hatA[a_idx] = processed_coeffs[bu_id * 2]
                hatA[a_idx_plus_t] = processed_coeffs[bu_id * 2 + 1]

    return hatA

In [59]:
qi = qp_partq[0]
COEF_a = NTT_parallel_16BU(EVAL_a, psi, qi)
pd.DataFrame(COEF_a)

,0
0,73284345390242
1,13920552779865
2,1846611669797
3,29285386033391
4,176825518443810
...,...
65531,154432294919593
65532,238641823902548
65533,134224460626353
65534,155105979739262
